In [1]:
from openai import OpenAI

# filepath: /path/to/your/file.py
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

In [3]:
client.files.create(
  file=open("../data/dad_joke_samples_train.jsonl", "rb"),
  purpose="fine-tune"
)

client.files.create(
  file=open("../data/dad_joke_samples_test.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-AkPWUJUXbsLLsdkETJrgW4', bytes=12094, created_at=1739193877, filename='dad_joke_samples_test.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [4]:
files = client.files.list(purpose="fine-tune")

for file in files:
    print(file.filename.upper(),':',file.id)

DAD_JOKE_SAMPLES_TEST.JSONL : file-AkPWUJUXbsLLsdkETJrgW4
DAD_JOKE_SAMPLES_TRAIN.JSONL : file-78aGFZduAzCr7gNpULVkpD
DAD_JOKE_REASONING_SAMPLES_TRAIN.JSONL : file-XZkorTai3cwNRvj4t3Eaap
DAD_JOKE_REASONING_SAMPLES_TEST.JSONL : file-1fwP5mJBAVSJm1enXNHvWC
DAD_JOKE_SAMPLES_TEST.JSONL : file-14gtA38iuJ7sJgqBmwsyNC
DAD_JOKE_SAMPLES_TRAIN.JSONL : file-Jp47nPA2Eq2S9p7VXCEsWz


In [5]:
client.fine_tuning.jobs.create(
  training_file="file-Jp47nPA2Eq2S9p7VXCEsWz",
  validation_file ="file-14gtA38iuJ7sJgqBmwsyNC",
  model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-1d4naLG5aPxm5KKXl4ZPVsp8', created_at=1739193895, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-RuBNDg5GaMqiVIMAjwK7becX', result_files=[], seed=1210483144, status='validating_files', trained_tokens=None, training_file='file-Jp47nPA2Eq2S9p7VXCEsWz', validation_file='file-14gtA38iuJ7sJgqBmwsyNC', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

In [8]:
import time


while True:
    events = client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-1d4naLG5aPxm5KKXl4ZPVsp8", limit=10)
    message = events.data[0].message
    print(message)
    if message == "The job has successfully completed":
        break
    

    time.sleep(20)

Step 131/144: training loss=0.54
Step 144/144: training loss=1.59, validation loss=1.58, full validation loss=1.90
Step 144/144: training loss=1.59, validation loss=1.58, full validation loss=1.90
Step 144/144: training loss=1.59, validation loss=1.58, full validation loss=1.90
Step 144/144: training loss=1.59, validation loss=1.58, full validation loss=1.90
The job has successfully completed


In [ ]:
# Retrieve the state of a fine-tune
dad_joke_model = client.fine_tuning.jobs.retrieve("ftjob-1d4naLG5aPxm5KKXl4ZPVsp").fine_tuned_model
print(dad_joke_model)

ft:gpt-4o-mini-2024-07-18:personal::AzOC0pEr
